<a href="https://colab.research.google.com/github/edierbravo/colab/blob/main/RED_NEURNONAL_CONVOLUCIONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam

In [41]:
# Cargar y preprocesar los datos
df = pd.read_csv('/content/dataset.csv')

# Codificar 'container' usando one-hot encoding
df_container_encoded = pd.get_dummies(df['container'], prefix='container')
df = pd.concat([df, df_container_encoded], axis=1)
df.drop('container', axis=1, inplace=True)

In [42]:
# Definir las etiquetas basadas en umbrales
def categorize_resource_state(row):
    cpu_usage = row['cpu-usage']
    memory_usage = row['memory-usage']
    cpu_limit = row['cpu-limit']
    memory_limit = row['memory-limit']
    cpu_request = row['cpu-request']
    memory_request = row['memory-request']

    # Convertir uso de CPU y memoria a porcentaje
    cpu_usage_percentage = cpu_usage / cpu_limit
    memory_usage_percentage = memory_usage / memory_limit

    # Porcentajes de solicitud
    cpu_request_percentage = row['cpu-request'] / cpu_limit
    memory_request_percentage = row['memory-request'] / memory_limit

    # Definir umbrales para clasificación
    if cpu_usage_percentage > 0.9 or memory_usage_percentage > 0.9 or cpu_request_percentage > 0.9 or memory_request_percentage > 0.9:
        return 'Crítico'
    elif cpu_usage_percentage > 0.7 or memory_usage_percentage > 0.7 or cpu_request_percentage > 0.7 or memory_request_percentage > 0.7:
        return 'Advertencia'
    else:
        return 'Normal'

df['state'] = df.apply(categorize_resource_state, axis=1)

# Codificar las etiquetas
labels = df['state'].unique()
df['state'] = df['state'].astype('category').cat.codes
num_classes = len(labels)

In [43]:
# Seleccionar características
features = df[['cpu-limit', 'cpu-request', 'cpu-usage', 'memory-limit', 'memory-request', 'memory-usage', 'receive-packets', 'receive-packets-dropped', 'total-packets', 'total-packets-dropped', 'transmit-packets', 'transmit-packets-dropped'] + list(df.filter(regex='^container_'))]
# Normalizar características
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [55]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features_scaled, df['state'], test_size=0.2, random_state=42)

# Convertir las etiquetas a formato categórico
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Definir el modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Primera capa oculta
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),  # Segunda capa oculta
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),  # Capa oculta final
    Dense(num_classes, activation='softmax')  # Capa de salida
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
# Compilar el modelo
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Usar EarlyStopping para evitar el sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [58]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9928 - loss: 0.0323 - val_accuracy: 0.9963 - val_loss: 0.0152
Epoch 2/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9940 - loss: 0.0250 - val_accuracy: 0.9969 - val_loss: 0.0144
Epoch 3/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9945 - loss: 0.0259 - val_accuracy: 0.9969 - val_loss: 0.0128
Epoch 4/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9966 - loss: 0.0124 - val_accuracy: 0.9969 - val_loss: 0.0143
Epoch 5/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9952 - loss: 0.0197 - val_accuracy: 0.9963 - val_loss: 0.0142
Epoch 6/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9964 - loss: 0.0154 - val_accuracy: 0.9969 - val_loss: 0.0111
Epoch 7/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9963 - loss: 0.0123 - val_accuracy: 0.9969 - val_loss: 0.0104
Epoch 8/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9972 - loss: 0.0122 - val_accuracy: 0.

In [59]:
# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9991 - loss: 0.0093
Test Accuracy: 0.9995


In [60]:
# Hacer predicciones
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Convertir etiquetas verdaderas a formato de clase
y_test_classes = np.argmax(y_test, axis=1)

# Mostrar el informe de clasificación
from sklearn.metrics import classification_report
print(classification_report(y_test_classes, predicted_classes, target_names=labels))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       512
 Advertencia       1.00      1.00      1.00       114
     Crítico       1.00      1.00      1.00      1382

    accuracy                           1.00      2008
   macro avg       1.00      1.00      1.00      2008
weighted avg       1.00      1.00      1.00      2008



In [72]:
# Definir las columnas en el mismo orden que durante el entrenamiento
feature_columns = [
    'cpu-limit', 'cpu-request', 'cpu-usage', 'memory-limit', 'memory-request',
    'memory-usage', 'receive-packets', 'receive-packets-dropped',
    'total-packets', 'total-packets-dropped', 'transmit-packets',
    'transmit-packets-dropped', 'container_amf1', 'container_amf2',
    'container_ausf', 'container_mongodb', 'container_nrf', 'container_nssf',
    'container_nwdaf-anlf', 'container_nwdaf-mtlf', 'container_pcf', 'container_smf1',
    'container_udm','container_udr', 'container_upf1'
]

# Supongamos que el modelo y el scaler ya están definidos y entrenados

# Datos del nuevo dato a probar
new_data = {
    'cpu-limit': 0.1,
    'cpu-request': 0.03,
    'cpu-usage': 0.0,
    'memory-limit': 500800900,
    'memory-request': 100800600,
    'memory-usage': 100300100,
    'receive-packets': 0.1,
    'receive-packets-dropped': 0,
    'total-packets': 0.1,
    'total-packets-dropped': 0,
    'transmit-packets': 0,
    'transmit-packets-dropped': 0,
    # Asegúrate de incluir columnas codificadas para 'container' si es necesario
    # Por ejemplo:
    # 'container_amf1': 1,
    # 'container_other': 0
    'container_amf1': 0,
    'container_amf2': 0,
    'container_ausf': 0,
    'container_mongodb': 0,
    'container_nrf': 0,
    'container_nssf': 0,
    'container_nwdaf-anlf':0,
    'container_nwdaf-mtlf':0,
    'container_pcf': 0,
    'container_smf1': 0,
    'container_udm': 0,
    'container_udr': 0,
    'container_upf1': 1
}

#  Crear un DataFrame para el nuevo dato con las columnas en el mismo orden
new_data_df = pd.DataFrame([new_data], columns=feature_columns)

In [73]:
# Normalizar el nuevo dato
new_data_scaled = scaler.fit_transform(new_data_df)
# new_data_scaled = scaler.transform(new_data_df)

# Hacer la predicción
prediction = model.predict(new_data_scaled)

# Obtener la clase predicha
predicted_class = np.argmax(prediction, axis=1)

# Convertir la clase predicha a una etiqueta
predicted_label = labels[predicted_class[0]]

print(f'Prediccion: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Prediccion: Crítico
